In [ ]:
# !pip install pandas
# !pip install numpy
# !pip install librosa
# !pip install tqdm

In [ ]:
import os
import numpy as np
import pandas as pd

from tqdm.auto import tqdm

# 경로 취합

In [ ]:
from pathlib import Path

text_base_dir = Path("Write your testset's text path")
audio_base_dir = Path("Write your testset's audio path")

# .txt 파일 glob 재설정 (재귀적 탐색)
labeled_data_list = sorted(text_base_dir.rglob("*.txt"))  # ✅ 모든 하위 폴더의 .txt 탐색
raw_data_list = sorted(audio_base_dir.rglob("*.wav"))     # ✅ 모든 하위 폴더의 .wav 탐색

print(f"텍스트 파일 수: {len(labeled_data_list)}")
print(f"오디오 파일 수: {len(raw_data_list)}")

텍스트 파일 수: 65123
오디오 파일 수: 65123


# 미인식 표현 삭제

In [11]:
deleted_count = 0
for txt_path in labeled_data_list:
    try:
        with txt_path.open("r", encoding="utf-8") as f:
            content = f.read()

        if "(())" in content:
            txt_path.unlink()
            deleted_count += 1

            # .wav 경로 계산
            relative_path = txt_path.relative_to(text_base_dir)
            wav_path = (audio_base_dir / relative_path).with_suffix(".wav")

            if wav_path.exists():
                wav_path.unlink()
                print(f"🗑️ 삭제된 .wav: {wav_path}")
            else:
                print(f"❓ 대응 .wav 없음: {wav_path}")
    except Exception as e:
        print(f"⚠️ 에러: {txt_path} — {e}")

print(f"\n✅ (()) 포함된 텍스트 삭제 개수: {deleted_count}")


🗑️ 삭제된 .wav: C:\Users\Playdata\Desktop\test\train_cut\D03_Audio\J14\S000001\0018.wav
🗑️ 삭제된 .wav: C:\Users\Playdata\Desktop\test\train_cut\D03_Audio\J14\S000002\0020.wav
🗑️ 삭제된 .wav: C:\Users\Playdata\Desktop\test\train_cut\D03_Audio\J14\S000002\0022.wav
🗑️ 삭제된 .wav: C:\Users\Playdata\Desktop\test\train_cut\D03_Audio\J14\S000003\0009.wav
🗑️ 삭제된 .wav: C:\Users\Playdata\Desktop\test\train_cut\D03_Audio\J14\S000006\0005.wav
🗑️ 삭제된 .wav: C:\Users\Playdata\Desktop\test\train_cut\D03_Audio\J14\S000006\0009.wav
🗑️ 삭제된 .wav: C:\Users\Playdata\Desktop\test\train_cut\D03_Audio\J14\S000007\0005.wav
🗑️ 삭제된 .wav: C:\Users\Playdata\Desktop\test\train_cut\D03_Audio\J14\S000010\0006.wav
🗑️ 삭제된 .wav: C:\Users\Playdata\Desktop\test\train_cut\D03_Audio\J14\S000010\0008.wav
🗑️ 삭제된 .wav: C:\Users\Playdata\Desktop\test\train_cut\D03_Audio\J14\S000012\0002.wav
🗑️ 삭제된 .wav: C:\Users\Playdata\Desktop\test\train_cut\D03_Audio\J14\S000012\0007.wav
🗑️ 삭제된 .wav: C:\Users\Playdata\Desktop\test\train_cut\D03_Audio\J

In [13]:
labeled_data_list = sorted(text_base_dir.rglob("*.txt"))  # ✅ 모든 하위 폴더의 .txt 탐색
raw_data_list = sorted(audio_base_dir.rglob("*.wav"))     # ✅ 모든 하위 폴더의 .wav 탐색

print(f"텍스트 파일 수: {len(labeled_data_list)}")
print(f"오디오 파일 수: {len(raw_data_list)}")

텍스트 파일 수: 62327
오디오 파일 수: 62327


In [14]:
# 최종 유효한 (wav, txt) 페어 저장 리스트
valid_audio_txt_pairs = []

for wav_path in raw_data_list:
    # txt 경로 유추
    relative_path = os.path.relpath(wav_path, audio_base_dir)
    txt_path = os.path.join(text_base_dir, relative_path).replace('.wav', '.txt')

    if os.path.exists(txt_path):
        valid_audio_txt_pairs.append((wav_path, txt_path))

print(f"✅ 1:1 매칭된 (wav, txt) 페어 수: {len(valid_audio_txt_pairs)}개")
print("예시:", valid_audio_txt_pairs[:3])

✅ 1:1 매칭된 (wav, txt) 페어 수: 62327개
예시: [(WindowsPath('C:/Users/Playdata/Desktop/test/train_cut/D03_Audio/J14/S000001/0001.wav'), 'C:\\Users\\Playdata\\Desktop\\test\\train_cut\\D03_Transcription\\J14\\S000001\\0001.txt'), (WindowsPath('C:/Users/Playdata/Desktop/test/train_cut/D03_Audio/J14/S000001/0002.wav'), 'C:\\Users\\Playdata\\Desktop\\test\\train_cut\\D03_Transcription\\J14\\S000001\\0002.txt'), (WindowsPath('C:/Users/Playdata/Desktop/test/train_cut/D03_Audio/J14/S000001/0003.wav'), 'C:\\Users\\Playdata\\Desktop\\test\\train_cut\\D03_Transcription\\J14\\S000001\\0003.txt')]


# txt 파일 내의 병기 표현 및 태그 삭제

In [ ]:
# 병기표현 및 태그 확인

import re
import os

def preview_bracket_and_speaker_tags(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        text = f.read()
    
    # 1. 병기 표현 패턴
    bracket_pattern = re.compile(r'\(\s*([^()]+?)\s*\)\s*/?\s*\(\s*([^()]+?)\s*\)')
    bracket_matches = list(bracket_pattern.finditer(text))

    # 2. 화자 태그 패턴
    #   - 띄어쓰기 있는 경우: ' o/'  or '\no/'
    #   - 붙어 있는 경우: '희망n/', '말씀b/' 등
    tag_pattern = re.compile(r'\b[a-zA-Z]/|[가-힣a-zA-Z]+[a-zA-Z]/')
    tag_matches = list(tag_pattern.finditer(text))

    if bracket_matches or tag_matches:
        print(f"\n📄 {os.path.basename(file_path)}:")

        if bracket_matches:
            print("  🔹 병기 표현 탐지:")
            for match in bracket_matches:
                original = match.group(0)
                left = match.group(1).strip()
                right = match.group(2).strip()

                # 앞이 영어 또는 숫자면 → 뒤 선택
                if re.fullmatch(r'[a-zA-Z0-9\s\-\.,&]+', left):
                    replacement = right
                else:
                    replacement = left

                print(f"   - 원문: {original} → 정제 결과: {replacement}")

        if tag_matches:
            print("  🔹 화자 태그 탐지:")
            for match in tag_matches:
                context = match.group(0)
                print(f"   - 발견된 태그: {context} (pos {match.start()})")

# ▶ 상위 10개 파일만 테스트 (필요시 수 조절)
for txt_path in labeled_data_list[:15]:
    preview_bracket_and_speaker_tags(txt_path)



📄 0007.txt:
  🔹 병기 표현 탐지:
   - 원문: (R&D)/(알앤디) → 정제 결과: 알앤디


In [ ]:
# 병기 표현과 태그 제거

def clean_text_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        text = f.read()

    def replace_variant(match):
        a = match.group(1).strip()
        b = match.group(2).strip()

        # 앞이 영어 또는 숫자일 경우 → 뒤 선택
        if re.fullmatch(r'[a-zA-Z0-9\s\-.,&]+', a):
            return b
        else:
            return a
        
    # 1. 병기 표현 제거: (A)/(B), (A) /(B), (A) (B) → 숫자가 있는 쪽 선택 (우선 B), 없으면 A        
    text = re.sub(
        r'\(\s*([^()]+?)\s*\)\s*/?\s*\(\s*([^()]+?)\s*\)',
        replace_variant,
        text
    )

    
    # 2. 화자 태그 제거: o/, n/, b/ 등 (띄어쓰기 유무 관계없이 제거)
    #    예: '희망n/' → '희망', 'b/' → '', ' n/' → ' '
    text = re.sub(r'\b[a-zA-Z]/', '', text)  # 독립된 태그 제거 (예: "b/")
    text = re.sub(r'([가-힣a-zA-Z])([a-zA-Z])/', r'\1', text)  # 단어에 붙은 태그 제거 (예: 희망n/ → 희망)

    with open(file_path, 'w', encoding='utf-8') as f:
        f.write(text)

# 모든 텍스트 파일 정제 적용
for txt_path in labeled_data_list:
    clean_text_file(txt_path)

print("✅ 모든 텍스트 파일 정제 완료")


✅ 모든 텍스트 파일 정제 완료


> 텍스트 데이터는 파일의 내용을 추출하여 데이터 프레임으로 만든다.

In [21]:
transcript_list = []
for labeled_data in tqdm(labeled_data_list):
    with open(labeled_data, 'r', encoding='UTF8') as f:
        line = f.readline()
        transcript_list.append(line)

df = pd.DataFrame(data=transcript_list, columns = ["transcript"])

  0%|          | 0/62327 [00:00<?, ?it/s]

In [22]:
# 텍스트 데이터로 만든 데이터프레임에 음성 파일 경로 컬럼을 추가
df["raw_data"] = raw_data_list

In [23]:
# Null data 유무 확인
df.isnull().values.sum()

np.int64(0)

In [24]:
print(df.shape)
df.head()

(62327, 2)


,transcript,raw_data
0,안녕하세요. 이번에 그 정보기술 전략에 지원한 지원자입니다.,C:\Users\Playdata\Desktop\test\train_cut\D03_A...
1,아 네 안녕하세요. 지원자님 어떤 일로 연락 주셨을까요.,C:\Users\Playdata\Desktop\test\train_cut\D03_A...
2,아 다름이 아니라 정보기술 전략팀이 뭔지 좀 더 상세히 좀 알고 싶어서요.,C:\Users\Playdata\Desktop\test\train_cut\D03_A...
3,아 네 정보기술 전략 직무에 대해 조금 더 자세히 알고 싶으시단 말씀이시죠.,C:\Users\Playdata\Desktop\test\train_cut\D03_A...
4,네 맞습니다.,C:\Users\Playdata\Desktop\test\train_cut\D03_A...


In [25]:
df.tail()

,transcript,raw_data
62322,총 37시간입니다.,C:\Users\Playdata\Desktop\test\train_cut\D03_A...
62323,예 알겠습니다.,C:\Users\Playdata\Desktop\test\train_cut\D03_A...
62324,네 그 외에 다른 문의사항 있으신가요?,C:\Users\Playdata\Desktop\test\train_cut\D03_A...
62325,아니 없습니다. 수고하세요.,C:\Users\Playdata\Desktop\test\train_cut\D03_A...
62326,네 다른 문의사항 있으시면 큐앤에이나 유선을 이용하여 질문 주시면 문의사항 도와드리...,C:\Users\Playdata\Desktop\test\train_cut\D03_A...


# check

In [26]:
idx = 1111

sentence = df.loc[idx, "transcript"]
audio_path = df.loc[idx, "raw_data"]

print(f"transcript : {sentence}")
print(f"audio_path : {audio_path}")

transcript : 네 전화 받았습니다.
audio_path : C:\Users\Playdata\Desktop\test\train_cut\D03_Audio\J14\S000055\0001.wav


In [28]:
# Librosa
import librosa
import IPython.display as ipd

# librosa load
wav, sr = librosa.load(audio_path, sr = None)

# Shape
print(f"wav.shape: {wav.shape}")
# 샘플링레이트
print(f"sampling_rate: {sr}")

# Original Mixed
ipd.display(ipd.Audio(wav, rate =sr))

wav.shape: (21712,)
sampling_rate: 8000


In [29]:
idx = 222

sentence = df.loc[idx, "transcript"]
audio_path = df.loc[idx, "raw_data"]

print(f"transcript : {sentence}")
print(f"audio_path : {audio_path}")

transcript : 네 그 지원자님이 말씀하신 사항은 운행 결과 평가보다는 그 운행 결과 보고 업무와 가까운 것으로 보여지세요.
audio_path : C:\Users\Playdata\Desktop\test\train_cut\D03_Audio\J14\S000012\0009.wav


In [30]:
# librosa load
wav, sr = librosa.load(audio_path, sr = None)

# Shape
print(f"wav.shape: {wav.shape}")
# 샘플링레이트
print(f"sampling_rate: {sr}")

# Original Mixed
ipd.display(ipd.Audio(wav, rate =sr))

wav.shape: (93280,)
sampling_rate: 8000


# 저장

In [ ]:
# 필요에 따라 데이터프레임을 csv 파일로 저장한다.

save_dir = Path("Write your path wanted to save")

df.to_csv(save_dir / "path_and_transcript_testset.csv", index= False)

In [ ]:
# csv 파일을 로드하여 df로 변환
df = pd.read_csv(save_dir / "path_and_transcript_testset.csv")

In [65]:
print(df.shape)
df.head()

(1855, 2)


,transcript,raw_data
0,여보세요.,/workspace/valid_dataset/D03_Audio/J13/S002202...
1,네 안녕하세요. 이제 서울시 에이아이 면접체험 운영 사무국입니다.,/workspace/valid_dataset/D03_Audio/J13/S002202...
2,지원자님 맞으신가요?,/workspace/valid_dataset/D03_Audio/J13/S002202...
3,아 네 안녕하세요. 저희 그 에이아이 검사 체험하신 건 컨설팅 동의 안내차 연락을...,/workspace/valid_dataset/D03_Audio/J13/S002202...
4,결과 분석에 대해서 따로 컨설팅을 받지 않으신 걸로 확인되시는데 저희가 곧 운영...,/workspace/valid_dataset/D03_Audio/J13/S002202...


In [66]:
df.shape

(1855, 2)

In [67]:
df.head()

,transcript,raw_data
0,여보세요.,/workspace/valid_dataset/D03_Audio/J13/S002202...
1,네 안녕하세요. 이제 서울시 에이아이 면접체험 운영 사무국입니다.,/workspace/valid_dataset/D03_Audio/J13/S002202...
2,지원자님 맞으신가요?,/workspace/valid_dataset/D03_Audio/J13/S002202...
3,아 네 안녕하세요. 저희 그 에이아이 검사 체험하신 건 컨설팅 동의 안내차 연락을...,/workspace/valid_dataset/D03_Audio/J13/S002202...
4,결과 분석에 대해서 따로 컨설팅을 받지 않으신 걸로 확인되시는데 저희가 곧 운영...,/workspace/valid_dataset/D03_Audio/J13/S002202...
